<a href="https://colab.research.google.com/github/BilalKhaliqWillis/BILAL-Assignment2/blob/main/BILAL_Assignment_12_Training_GPU_for_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Training on GPU
# Importing Required Libraries
import time
import os
import numpy as np
import tensorflow as tf
from scipy.io import loadmat
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [3]:
# Downloading the SVHN Dataset
if not os.path.isfile('train_32x32.mat'):
    !wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
    !wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat
else:
    print("Dataset already downloaded")

--2026-02-21 20:11:29--  http://ufldl.stanford.edu/housenumbers/train_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 182040794 (174M) [text/plain]
Saving to: ‘train_32x32.mat’

train_32x32.mat     100%[===================>] 173.61M  37.1MB/s    in 9.6s    

2026-02-21 20:11:39 (18.1 MB/s) - ‘train_32x32.mat’ saved [182040794/182040794]

--2026-02-21 20:11:39--  http://ufldl.stanford.edu/housenumbers/test_32x32.mat
Resolving ufldl.stanford.edu (ufldl.stanford.edu)... 171.64.68.10
Connecting to ufldl.stanford.edu (ufldl.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64275384 (61M) [text/plain]
Saving to: ‘test_32x32.mat’

test_32x32.mat      100%[===================>]  61.30M  13.6MB/s    in 6.9s    

2026-02-21 20:11:46 (8.94 MB/s) - ‘test_32x32.mat’ saved [64275384/64

In [4]:
# Loading and Preprocessing the Data
train = loadmat('train_32x32.mat')
test = loadmat('test_32x32.mat')

x_train = train['X'] / 255.0
y_train = train['y']
x_test = test['X'] / 255.0
y_test = test['y']

# Converting label 10 to 0
y_train[y_train == 10] = 0
y_test[y_test == 10] = 0

# Converting RGB to grayscale
x_train = np.mean(x_train, axis=2, keepdims=True).transpose(3,0,1,2)
x_test = np.mean(x_test, axis=2, keepdims=True).transpose(3,0,1,2)

print("Training shape:", x_train.shape)
print("Testing shape:", x_test.shape)

Training shape: (73257, 32, 32, 1)
Testing shape: (26032, 32, 32, 1)


In [5]:
# Defining CNN Model
def get_cnn_model(input_shape):
    model = Sequential([
        Conv2D(8, (3,3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((3,3)),
        Conv2D(16, (3,3), activation='relu', padding='same'),
        MaxPooling2D((3,3)),
        Flatten(),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [6]:
# Training on GPU
model_gpu = get_cnn_model(x_train.shape[1:])
early_stop = EarlyStopping(monitor='loss', patience=3)

start_time = time.time()

history_gpu = model_gpu.fit(
    x_train, y_train,
    epochs=3,
    batch_size=128,
    validation_split=0.15,
    callbacks=[early_stop]
)

gpu_time = time.time() - start_time
print(f"GPU Training Time: {gpu_time:.2f} seconds")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3
487/487 ━━━━━━━━━━━━━━━━━━━━ 26s 49ms/step - accuracy: 0.2015 - loss: 2.2124 - val_accuracy: 0.4661 - val_loss: 1.6813
Epoch 2/3
487/487 ━━━━━━━━━━━━━━━━━━━━ 23s 48ms/step - accuracy: 0.4571 - loss: 1.6044 - val_accuracy: 0.6446 - val_loss: 1.1716
Epoch 3/3
487/487 ━━━━━━━━━━━━━━━━━━━━ 41s 49ms/step - accuracy: 0.5765 - loss: 1.2731 - val_accuracy: 0.6951 - val_loss: 1.0176
GPU Training Time: 90.87 seconds


In [7]:
# Evaluating GPU Model
gpu_loss, gpu_acc = model_gpu.evaluate(x_test, y_test, verbose=2)
print("GPU Test Accuracy:", gpu_acc)

814/814 - 10s - 13ms/step - accuracy: 0.7021 - loss: 1.0121
GPU Test Accuracy: 0.7020590305328369
